### Quick Start

Let’s take a very basic configuration file that looks like this:

The structure of INI files is described in the following section. Essentially, the file consists of sections, each of which contains keys with values. configparser classes can read and write such files. Let’s start by creating the above configuration file programmatically.

In [2]:
import configparser

config = configparser.ConfigParser()

config['DEFAULT'] = {'ServerAliveInterval': '45',

                     'Compression': 'yes',

                     'CompressionLevel': '9'}

config['forge.example'] = {}

config['forge.example']['User'] = 'hg'

config['topsecret.server.example'] = {}

topsecret = config['topsecret.server.example']

topsecret['Port'] = '50022'     # mutates the parser

topsecret['ForwardX11'] = 'no'  # same here

config['DEFAULT']['ForwardX11'] = 'yes'

with open('config/example.ini', 'w') as configfile:

  config.write(configfile)


As you can see, we can treat a config parser much like a dictionary. There are differences, outlined later, but the behavior is very close to what you would expect from a dictionary.

Now that we have created and saved a configuration file, let’s read it back and explore the data it holds.

In [3]:
config = configparser.ConfigParser()
config.sections()

[]

In [5]:
config.read('config/example.ini')

['config/example.ini']

In [6]:
config.sections()

['forge.example', 'topsecret.server.example']

In [8]:
'forge.example' in config

True

In [9]:
'python.org' in config

False

In [10]:
config['forge.example']['User']

'hg'

In [11]:
config['DEFAULT']['Compression']

'yes'

In [12]:
topsecret = config['topsecret.server.example']
topsecret['ForwardX11']

'no'

In [13]:
topsecret['Port']

'50022'

In [14]:
for key in config['forge.example']:  
    print(key)

user
serveraliveinterval
compression
compressionlevel
forwardx11


In [15]:
config['forge.example']['ForwardX11']

'yes'

As we can see above, the API is pretty straightforward. The only bit of magic involves the DEFAULT section which provides default values for all other sections 1. Note also that keys in sections are case-insensitive and stored in lowercase 1.

It is possible to read several configurations into a single ConfigParser, where the most recently added configuration has the highest priority. Any conflicting keys are taken from the more recent configuration while the previously existing keys are retained.

In [18]:
another_config = configparser.ConfigParser()

another_config.read('config/example.ini')

['config/example.ini']

In [19]:
another_config['topsecret.server.example']['Port']

'50022'

In [20]:
another_config.read_string("[topsecret.server.example]\nPort=48484")

another_config['topsecret.server.example']['Port']

'48484'

In [21]:
another_config.read_dict({"topsecret.server.example": {"Port": 21212}})

another_config['topsecret.server.example']['Port']

'21212'

In [22]:
another_config['topsecret.server.example']['ForwardX11']

'no'

This behaviour is equivalent to a ConfigParser.read() call with several files passed to the filenames parameter.

### Supported Datatypes

Config parsers do not guess datatypes of values in configuration files, always storing them internally as strings. This means that if you need other datatypes, you should convert on your own:

In [23]:
int(topsecret['Port'])

50022

In [24]:
float(topsecret['CompressionLevel'])

9.0

Since this task is so common, config parsers provide a range of handy getter methods to handle integers, floats and booleans. The last one is the most interesting because simply passing the value to bool() would do no good since `bool('False')` is still `True`. This is why config parsers also provide `getboolean()`. This method is case-insensitive and recognizes Boolean values from `'yes'/'no'`, `'on'/'off'`, `'true'/'false'` and `'1'/'0'`. For example:

In [25]:
topsecret.getboolean('ForwardX11')

False

In [26]:
config['forge.example'].getboolean('ForwardX11')

True

In [27]:
config.getboolean('forge.example', 'Compression')

True

In [28]:
topsecret.get('Port')

'50022'

In [29]:
topsecret.get('CompressionLevel')

'9'

In [30]:
topsecret.get('Cipher')

In [31]:
topsecret.get('Cipher', '3des-cbc')

'3des-cbc'

Please note that default values have precedence over fallback values. For instance, in our example the `'CompressionLevel'` key was specified only in the `'DEFAULT'` section. If we try to get it from the section `'topsecret.server.example'`, we will always get the default, even if we specify a fallback:

In [32]:
topsecret.get('CompressionLevel', '3')

'9'

One more thing to be aware of is that the parser-level get() method provides a custom, more complex interface, maintained for backwards compatibility. When using this method, a fallback value can be provided via the `fallback` keyword-only argument:

In [33]:
'BatchMode' in topsecret

False

In [34]:
topsecret.getboolean('BatchMode', fallback=True)

True

In [35]:
config['DEFAULT']['BatchMode'] = 'no'

In [36]:
topsecret.getboolean('BatchMode', fallback=True)

False

## Supported INI File Structure

A configuration file consists of sections, each led by a `[section]` header, followed by key/value entries separated by a specific string (`=` or `:` by default). By default, section names are case sensitive but keys are not. Leading and trailing whitespace is removed from keys and values. Values can be omitted if the parser is configured to allow it, in which case the key/value delimiter may also be left out. Values can also span multiple lines, as long as they are indented deeper than the first line of the value. Depending on the parser’s mode, blank lines may be treated as parts of multiline values or ignored.

By default, a valid section name can be any string that does not contain `‘\n’` or `‘]’`. To change this, see ConfigParser.SECTCRE.

Configuration files may include comments, prefixed by specific characters (`#` and `;` by default). Comments may appear on their own on an otherwise empty line, possibly indented. 

For example: